In [43]:
# Imports
import csv
import pandas as pd
import nltk
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
import time


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [0]:
# Lendo o CSV
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_01/master/output/results.csv')
colecao = colecao.astype(str)

# Laboratório 04: Indexação e Processamento de Consulta

# Questão 1

Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [45]:
tokens = []
indices = {}
n = 0

tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese')

# Indexacao
for texto in colecao.texto:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]
  tokens.extend(palavras)
  n += 1
  for t in set(palavras):
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)

{'discurso': [1, 7, 15, 22, 24, 26, 56, 61, 62, 64, 66, 70, 76, 79, 80, 88, 103, 104, 105, 110, 113, 122, 138, 141, 152, 153, 154, 161, 163, 164, 170, 174, 187, 193, 195, 197, 205, 209, 218, 219, 229, 231, 245, 266, 269, 272], 'propagação': [1, 32, 254], 'duplicação': [1, 232], 'equivale': [1, 110, 208], 'cinco': [1, 16, 36, 46, 53, 81, 82, 100, 104, 108, 114, 146, 148, 158, 162, 165, 170, 173, 174, 199, 201, 203, 205, 208, 214, 215, 224, 225, 234, 248, 253, 255, 265, 266, 269], 'sofre': [1, 32, 84, 153, 154], 'humanitária': [1, 6, 24, 26, 100, 136, 150, 153, 154, 156, 176, 187], 'mostram': [1, 4, 24, 27, 68, 94, 95, 143, 193, 204, 236, 263], 'hospitais': [1, 32, 63, 68, 247], 'ajuda': [1, 24, 32, 63, 126, 134, 136, 152, 160, 183, 187, 254, 264, 266], 'precisam': [1, 12, 61, 77, 118, 128, 158, 187, 193, 254, 263, 264], 'estado': [1, 8, 10, 12, 13, 22, 24, 27, 28, 31, 32, 33, 34, 48, 52, 55, 56, 63, 66, 69, 72, 76, 77, 81, 85, 86, 87, 88, 94, 96, 100, 103, 104, 118, 119, 126, 130, 136, 

In [0]:
# Guardando o indice em formato CSV
inverted_index_df = pd.DataFrame.from_dict(indices, orient="index")
inverted_index_df.to_csv('data.csv')
indices = indices

# Questão 2

Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez.


1.   Defina 5 consultas de um termo somente.
2.   Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.   Dê evidências de que sua implementação está correta.
4.   Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [48]:
# Funcao que implementa a consulta documento-por-vez
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(palavras)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i == documento): #i[0]
                    sd += i #i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)
 
# Exemplo
documentoPorVez("bolsonaro", indices, 10)  

[(110, 110),
 (109, 109),
 (107, 107),
 (106, 106),
 (105, 105),
 (104, 104),
 (103, 103),
 (102, 102),
 (101, 101),
 (100, 100)]

In [49]:
# Funcao que implementa a consulta termo-por-vez    
def termoPorVez(consulta, indices, k):
    a = {}
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for lista_invertida in listas_invertidas:
        for p in lista_invertida:
            d = p
            freq = p
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
    return heapq.nlargest(k, r)
  
# Exemplo  
termoPorVez("bolsonaro", indices, 10)

[(277, 277),
 (270, 270),
 (268, 268),
 (266, 266),
 (261, 261),
 (258, 258),
 (256, 256),
 (254, 254),
 (252, 252),
 (249, 249)]

In [0]:
 # Definindo consultas
  
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

In [0]:
# Executando as consultas 

resultados_documento = []
resultados_termo = []
resultados_documento_tempo = []
resultados_termo_tempo = []
k = 10

for consulta in consultas:
  inicio_documento = time.time()
  resultados_documento.append(documentoPorVez(consulta, indices, k))
  fim_documento = time.time()
  resultados_documento_tempo.append(fim_documento - inicio_documento)
  
  inicio_termo = time.time()
  resultados_termo.append(termoPorVez(consulta, indices, k))
  fim_termo = time.time()
  resultados_termo_tempo.append(fim_termo - inicio_termo)


In [52]:
# Criando e imprimindo a tabela com os dados dos documentos

tabela_consultas = pd.DataFrame()
tabela_consultas['consulta'] = consultas
tabela_consultas['documento_por_vez'] = resultados_documento
tabela_consultas['termo_por_vez'] = resultados_termo
tabela_consultas['comparacao'] = tabela_consultas.documento_por_vez == tabela_consultas.termo_por_vez
tabela_consultas

,consulta,documento_por_vez,termo_por_vez,comparacao
0,presidente,"[(110, 110), (109, 109), (107, 107), (106, 106...","[(277, 277), (276, 276), (275, 275), (274, 274...",False
1,educação,"[(108, 108), (89, 89), (62, 62), (58, 58), (22...","[(259, 259), (249, 249), (240, 240), (219, 219...",False
2,privatização,"[(62, 62)]","[(245, 245), (221, 221), (204, 204), (184, 184...",False
3,empresa,"[(107, 107), (103, 103), (95, 95), (85, 85), (...","[(272, 272), (262, 262), (257, 257), (234, 234...",False
4,bolsonaro,"[(110, 110), (109, 109), (107, 107), (106, 106...","[(277, 277), (270, 270), (268, 268), (266, 266...",False


In [53]:
# Comparando os tempos medios de de execucao e uso de memoria

tabela_tempo = pd.DataFrame()
tabela_tempo['tempo_médio_documento_por_vez'] = resultados_documento_tempo
tabela_tempo['tempo_médio_termo_por_vez'] = resultados_termo_tempo
tabela_tempo

,tempo_médio_documento_por_vez,tempo_médio_termo_por_vez
0,0.000538,0.000236
1,0.000133,0.000043
2,0.000051,0.000007
3,0.000150,0.000033
4,0.000477,0.000208


# Questão 3

Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

1.  Defina 5 consultas conjuntivas (AND).
2.  Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.  Dê evidências de que sua implementação está correta.

In [0]:
# Funcao que implementa a consulta conjuntiva (AND)

def consulta_and(consulta, indices, k):
  
    indices_consulta = []
    ranking = []
  
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            indices_consulta.append(indices[palavra])
  
    todas_as_consultas = [item for sublista in indices_consulta for item in sublista]
    todas_as_consultas.sort()
  
    for i in range(len(todas_as_consultas)):
        score_documento = 0
    d = todas_as_consultas.pop()
    for indice in todas_as_consultas:
        if indice == d: #indice[0] e d[0]
            score_documento += indice[1] + d[1]
        if score_documento != 0:
            heapq.heappush(rank, (score_documento, d[0]))
        
    return heapq.nlargest(k, ranking)

In [0]:
# Definindo consultas

consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

In [59]:
# Criando e imprimindo a tabela com os dados dos documentos

resultados_and = []
k = 10

for consulta in consultas:
    resultados_and.append(consulta_and(consulta, indices, k))


tabela_and = pd.DataFrame()
tabela_and['consulta'] = consultas
tabela_and['AND'] = resultados_and
tabela_and

,consulta,AND
0,presidente,[]
1,educação,[]
2,privatização,[]
3,empresa,[]
4,bolsonaro,[]
